# Start H2O

In [1]:
# Start and connect to a local H2O cluster
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_72"; Java(TM) SE Runtime Environment (build 1.8.0_72-b15); Java HotSpot(TM) 64-Bit Server VM (build 25.72-b15, mixed mode)
  Starting server from /Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpx7q7ux_8
  JVM stdout: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpx7q7ux_8/h2o_jofaichow_started_from_python.out
  JVM stderr: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpx7q7ux_8/h2o_jofaichow_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.1
H2O cluster version age:,"7 days, 3 hours and 17 minutes"
H2O cluster name:,H2O_from_python_jofaichow_8jp16u
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# Import Data

In [2]:
# Import house price data from GitHub
train = h2o.import_file("https://github.com/woobe/h2o_training_2018_02/raw/master/examples/clustering/water_treatment_plant.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
# Alternatively, import data locally
# train = h2o.import_file("water_treatment_plant.csv")

In [4]:
# Have a quick look
train.head(5)

name,Q-E,ZN-E,PH-E,DBO-E,DQO-E,SS-E,SSV-E,SED-E,COND-E,PH-P,DBO-P,SS-P,SSV-P,SED-P,COND-P,PH-D,DBO-D,DQO-D,SS-D,SSV-D,SED-D,COND-D,PH-S,DBO-S,DQO-S,SS-S,SSV-S,SED-S,COND-S,RD-DBO-P,RD-SS-P,RD-SED-P,RD-DBO-S,RD-DQO-S,RD-DBO-G,RD-DQO-G,RD-SS-G,RD-SED-G
D-1/3/90,44101,1.5,7.8,nan,407,166,66.3,4.5,2110,7.9,nan,228,70.2,5.5,2120,7.9,nan,280,94,72.3,0.3,2010,7.3,nan,84,21,81,0.02,2000,nan,58.8,95.5,nan,70,nan,79.4,87.3,99.6
D-2/3/90,39024,3,7.7,nan,443,214,69.2,6.5,2660,7.7,nan,244,75.4,7.7,2570,7.6,nan,474,96,79.2,0.4,2700,7.5,nan,91,17,94.1,0,2590,nan,60.7,94.8,nan,80.8,nan,79.5,92.1,100
D-4/3/90,32229,5,7.6,nan,528,186,69.9,3.4,1666,7.7,nan,220,72.7,4.5,1594,7.7,nan,272,92,78.3,0.2,1742,7.6,nan,128,21,81,0.05,1888,nan,58.2,95.6,nan,52.9,nan,75.8,88.7,98.5
D-5/3/90,35023,3.5,7.9,205,588,192,65.6,4.5,2430,7.8,236,268,73.1,8.5,2280,7.8,158,376,96,77.1,0.4,2060,7.6,20,104,20,96.7,0,1840,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100
D-6/3/90,36924,1.5,8,242,496,176,64.8,4,2110,7.9,nan,236,57.6,4.5,2020,7.8,nan,372,88,68.2,0.2,2250,7.6,19,108,22,65.9,0.02,2120,nan,62.7,95.6,nan,71,92.1,78.2,87.5,99.5


# Define Features

In [5]:
# Define features (or predictors)
features = list(train.columns) # we want to use all the information
features.remove('name')    # we need to exclude the target 'medv' (otherwise there is nothing to predict)
features

['Q-E',
 'ZN-E',
 'PH-E',
 'DBO-E',
 'DQO-E',
 'SS-E',
 'SSV-E',
 'SED-E',
 'COND-E',
 'PH-P',
 'DBO-P',
 'SS-P',
 'SSV-P',
 'SED-P',
 'COND-P',
 'PH-D',
 'DBO-D',
 'DQO-D',
 'SS-D',
 'SSV-D',
 'SED-D',
 'COND-D',
 'PH-S',
 'DBO-S',
 'DQO-S',
 'SS-S',
 'SSV-S',
 'SED-S',
 'COND-S',
 'RD-DBO-P',
 'RD-SS-P',
 'RD-SED-P',
 'RD-DBO-S',
 'RD-DQO-S',
 'RD-DBO-G',
 'RD-DQO-G',
 'RD-SS-G',
 'RD-SED-G']

# k-Means Clustering

In [6]:
from h2o.estimators.kmeans import H2OKMeansEstimator
km = H2OKMeansEstimator(k = 10, init="Random", seed=1234, standardize=True, estimate_k = True)
km.train(x = features, training_frame = train)

kmeans Model Build progress: |████████████████████████████████████████████| 100%


In [7]:
km

Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1519109639712_1


ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 15449.19045661804
Total Sum of Square Error to Grand Mean: 19397.00001397763
Between Cluster Sum of Square Error: 3947.809557359591
Centroid Statistics: 


,centroid,size,within_cluster_sum_of_squares
,1.0,268.0,6854.5368882
,2.0,252.0,7404.9121991
,3.0,7.0,1189.7413693


Scoring History: 


,timestamp,duration,iterations,number_of_clusters,number_of_reassigned_observations,within_cluster_sum_of_squares
,2018-02-20 07:54:04,0.017 sec,0.0,0.0,nan,nan
,2018-02-20 07:54:04,0.049 sec,1.0,1.0,527.0,42967.5294213
,2018-02-20 07:54:04,0.063 sec,2.0,1.0,0.0,19397.0000000
,2018-02-20 07:54:04,0.082 sec,3.0,2.0,108.0,18219.0220275
,2018-02-20 07:54:04,0.084 sec,4.0,2.0,58.0,17607.0174138
,2018-02-20 07:54:04,0.087 sec,5.0,2.0,31.0,17416.9116807
,2018-02-20 07:54:04,0.089 sec,6.0,2.0,15.0,17350.4792475
,2018-02-20 07:54:04,0.091 sec,7.0,2.0,13.0,17328.0444634
,2018-02-20 07:54:04,0.093 sec,8.0,2.0,12.0,17304.3460203
,2018-02-20 07:54:04,0.095 sec,9.0,2.0,9.0,17284.5100055


In [8]:
km.predict(train)

kmeans prediction progress: |█████████████████████████████████████████████| 100%


predict
1
1
1
1
1
1
1
1
0
1
